In [1]:
import pandas as pd

combined_data = pd.read_csv("../data/ChemBL2599_combined_data.csv.gz", low_memory=False)
combined_data.shape

(7572, 251)

In [3]:
# Remove non numerical unused attributes
machine_learning_df = combined_data.drop(columns=["Molecule ChEMBL ID", "Molecule Name", "Molecule Max Phase", "#RO5 Violations", 
                                                  "Compound Key", "Smiles", "Standard Type", "Standard Relation", "Standard Value",	"Standard Units", 
                                                  "pChEMBL Value", "Data Validity Comment",	"Comment", "Uo Units", "Ligand Efficiency BEI",	"Ligand Efficiency LE",
                                                    "Ligand Efficiency LLE", "Ligand Efficiency SEI", "Potential Duplicate", "Assay ChEMBL ID", "Assay Description",
                                                    "Assay Type", "BAO Format ID", "BAO Label", "Assay Organism", "Assay Tissue ChEMBL ID", "Assay Tissue Name",	
                                                    "Assay Cell Type", "Assay Subcellular Fraction", "Assay Parameters", "Assay Variant Accession", 
                                                    "Assay Variant Mutation", "Target ChEMBL ID", "Target Name", "Target Organism", "Target Type", "Document ChEMBL ID",
                                                    "Source ID", "Source Description", "Document Journal", "Document Year", "Cell ChEMBL ID", "Properties", "Action Type",
                                                    "Standard Text Value", "Value", "IC50_m", "Converted Smiles"])
machine_learning_df.head()

,Molecular Weight,AlogP,pIC50_m,converted_smile_0,converted_smile_1,converted_smile_2,converted_smile_3,converted_smile_4,converted_smile_5,converted_smile_6,...,converted_smile_190,converted_smile_191,converted_smile_192,converted_smile_193,converted_smile_194,converted_smile_195,converted_smile_196,converted_smile_197,converted_smile_198,converted_smile_199
0,369.47,2.52,6.499997,1.854377,808.395217,19.388541,15.774469,15.774469,12.935561,9.373808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.614840
1,355.45,2.01,7.500038,1.858084,808.121772,18.681434,15.119768,15.119768,12.435561,8.847099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.644743
2,375.86,2.36,6.799998,1.858084,813.693554,18.681434,14.497733,15.253662,12.435561,8.536081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.630094
3,367.46,1.88,6.599998,1.582716,853.946965,18.802754,15.241088,15.241088,13.097357,9.554206,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.634156
4,359.39,3.25,6.657577,1.610247,1135.828414,18.802754,14.523392,14.523392,13.097357,8.296359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.433853


In [16]:
from abc import ABC, abstractmethod
from typing import Iterable, Union
from sklearn.utils.validation import check_array, column_or_1d
from inspect import isclass
from typing import List, Tuple, Union

from sklearn.exceptions import NotFittedError


def check_is_fitted(applicability_domain,
                    attributes: Union[str, List[str], Tuple[str]],
                    msg: str = None,
                    all_or_any=all):

    if isclass(applicability_domain):
        raise TypeError("{} is a class, not an instance.".format(applicability_domain))
    if msg is None:
        msg = (
            "This %(name)s instance is not fitted yet. Call 'fit' with "
            "appropriate arguments before using this applicability domain."
        )
    if not hasattr(applicability_domain, "fit"):
        raise TypeError("%s is not an estimator instance." % (applicability_domain))
    if not isinstance(attributes, (list, tuple)):
        attributes = [attributes]
    is_fitted = all_or_any([hasattr(applicability_domain, attr) for attr in attributes])
    if not is_fitted:
        raise NotFittedError(msg % {"name": type(applicability_domain).__name__})

class ApplicabilityDomain(ABC):
    def __init__(self):
        self.fitted_ = False

    def fit(self, X):
        X = check_array(X)
        self.num_points, self.num_dims = X.shape
        self._fit(X)
        self.fitted_ = True

    @abstractmethod
    def _fit(self, X):
        pass

    def contains(self, sample) -> Union[bool, Iterable[bool]]:
        check_is_fitted(self, 'fitted_')
        try:
            sample = column_or_1d(sample)
        except ValueError:
            sample = check_array(sample, accept_large_sparse=False)
        if sample.ndim == 1 and sample.shape[0] != self.num_dims:
            raise ValueError('sample must have the same number of features as the applicability domain; '
                             f'{sample.shape[0]} and {self.num_dims} respectively')
        elif sample.ndim == 2 and sample.shape[1] != self.num_dims:
            raise ValueError('sample must have the same number of features as the applicability domain; '
                             f'{sample.shape[1]} and {self.num_dims} respectively')
        return self._contains(sample)

    @abstractmethod
    def _contains(self, sample):
        pass

In [ ]:
from math import floor
from typing import Union, Tuple, Optional
import numpy as np
import scipy
from numpy.random import RandomState
from scipy.spatial.distance import cdist, _METRICS as dist_fns
from scipy.stats import f as Fdistrib
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, NearestNeighbors
from sklearn.neighbors._kde import KernelDensity
from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.utils.extmath import stable_cumsum

class TopKatApplicabilityDomain(ApplicabilityDomain):
    def __init__(self):
        super().__init__()

    def _fit(self, X):
        self.X_min_, self.X_max_ = X.min(axis=0), X.max(axis=0)
        S = (2 * X - self.X_max_ - self.X_min_) / np.where((self.X_max_ - self.X_min_) != 0,
                                                           (self.X_max_ - self.X_min_),1)
        S = np.c_[np.ones(S.shape[0]), S]
        self.eigen_val, self.eigen_vec = np.linalg.eig(S.T.dot(S))
        self.eigen_val, self.eigen_vec = np.real(self.eigen_val), np.real(self.eigen_vec)
        OPS = S.dot(self.eigen_vec)
        self.OPS_min_ = OPS.min(axis=0)
        self.OPS_max_ = OPS.max(axis=0)

    def _contains(self, sample):
        Ssample = (2 * sample - self.X_max_ - self.X_min_) / np.where((self.X_max_ - self.X_min_) != 0,
                                                                      (self.X_max_ - self.X_min_),1)
        if sample.ndim == 1:
            Ssample = np.c_[1, Ssample.reshape((1, -1))]
        else:
            Ssample = np.c_[np.ones((sample.shape[0], 1)), Ssample]
        OPS_sample = Ssample.dot(self.eigen_vec)
        denom = np.divide(np.ones_like(self.eigen_val, dtype=float),
                          self.eigen_val,
                          out=np.zeros_like(self.eigen_val),
                          where=self.eigen_val!=0)
        dOPS = (OPS_sample * OPS_sample).dot(denom)
        if sample.ndim == 1 and isinstance(dOPS, np.ndarray):
            dOPS = dOPS.item()
        return dOPS < (5 * (self.num_dims)) / (2 * self.num_points)


class LeverageApplicabilityDomain(ApplicabilityDomain):
    def __init__(self):
        super().__init__()
        self.scaler = StandardScaler()

    def _fit(self, X):
        X = self.scaler.fit_transform(X)
        self.var_covar = np.linalg.inv(X.T.dot(X))
        self.threshold = 3 * (self.num_dims + 1) / self.num_points

    def _contains(self, sample):
        if sample.ndim == 1:
            sample = self.scaler.transform(sample.reshape(1, -1))
            h = sample.dot(self.var_covar).dot(sample.T)
        else:
            sample = self.scaler.transform(sample)
            h = np.diag(sample.dot(self.var_covar).dot(sample.T))
        return h < self.threshold

class KNNApplicabilityDomain(ApplicabilityDomain):
    def __init__(self, k: int = 5,
                 alpha: float = 0.95,
                 hard_threshold: float = None,
                 scaling: Optional[str] = 'robust',
                 dist: str = 'euclidean',
                 scaler_kwargs=None,
                 njobs: int=1):
        super().__init__()
        if scaler_kwargs is None:
            scaler_kwargs = {}
        if alpha > 1 or alpha < 0:
            raise ValueError('alpha must lie between 0 and 1')
        scaling_methods = ('robust', 'minmax', 'maxabs', 'standard', None)
        if scaling not in scaling_methods:
            raise ValueError(f'scaling method must be one of {scaling_methods}')
        if scaling == 'robust':
            self.scaler = RobustScaler(**scaler_kwargs)
        elif scaling == 'minmax':
            self.scaler = MinMaxScaler(**scaler_kwargs)
        elif scaling == 'maxabs':
            self.scaler = MaxAbsScaler(**scaler_kwargs)
        elif scaling == 'standard':
            self.scaler = StandardScaler(**scaler_kwargs)
        elif scaling is None:
            self.scaler = None
        else:
            raise NotImplementedError('scaling method not implemented')
        if dist not in dist_fns.keys():
            raise NotImplementedError('distance type is not available')
        else:
            self.dist = dist
        self.k = k
        self.alpha = alpha
        self.hard_threshold = hard_threshold
        self.nn = NearestNeighbors(n_neighbors=k, metric=dist, n_jobs=njobs)

    def _fit(self, X):
        self.X_norm = self.scaler.fit_transform(X) if self.scaler is not None else X
        self.nn.fit(self.X_norm)
        self.kNN_dist = self.nn.kneighbors(self.X_norm, return_distance=True, n_neighbors=self.k+1)[0][:, 1:].mean(axis=1)
        kNN_train_distance_sorted_ = np.trim_zeros(np.sort(self.kNN_dist))
        if self.hard_threshold:
            self.threshold_ = self.hard_threshold
        else:
            self.threshold_ = kNN_train_distance_sorted_[floor(kNN_train_distance_sorted_.shape[0] * self.alpha) - 1]
        return self

    def _contains(self, sample):
        if self.scaler is not None:
            if sample.ndim == 1:
                sample = self.scaler.transform(sample.reshape((1, len(sample))))
            else:
                sample = self.scaler.transform(sample)
        kNN_sample_dist = self.nn.kneighbors(sample, return_distance=True)[0].mean(axis=1)
        norm_dist = kNN_sample_dist / self.threshold_
        if self.hard_threshold:
            return norm_dist < 1
        return norm_dist <= 1

In [7]:
import keras_tuner as kt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow import keras

X = machine_learning_df.drop(columns=["pIC50_m"]).values
y = machine_learning_df["pIC50_m"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_valid, X_train_sub = X_train[:500], X_train[500:]
y_valid, y_train_sub = y_train[:500], y_train[500:]

def create_model(neurons1, neurons2, neurons3, lr, dropout_rate):
    model = keras.models.Sequential([
        keras.layers.Dense(neurons1, activation="relu", input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(neurons2, activation="relu"),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(neurons3, activation="relu"),
        keras.layers.Dropout(dropout_rate),
        keras.layers.Dense(1)
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=["mse"]
    )
    return model

def build_model(hp):
    neurons1 = hp.Int("neurons1", min_value=32, max_value=256, step=32)
    neurons2 = hp.Int("neurons2", min_value=16, max_value=128, step=16)
    neurons3 = hp.Int("neurons3", min_value=8, max_value=64, step=8)
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    dropout_rate = hp.Float("dropout_rate", min_value=0.0, max_value=0.5, step=0.05)
    return create_model(neurons1, neurons2, neurons3, lr, dropout_rate)

tuner = kt.BayesianOptimization(
    build_model,
    objective='val_mse',
    max_trials=60,
    overwrite=True
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=10,
    restore_best_weights=True
)

tuner.search(
    X_train_sub, y_train_sub,
    epochs=100,
    validation_data=(X_valid, y_valid),
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

best_model = tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

print("Best hyperparameters:")
print(f"neurons1: {best_hps.get('neurons1')}")
print(f"neurons2: {best_hps.get('neurons2')}")
print(f"neurons3: {best_hps.get('neurons3')}")
print(f"learning_rate: {best_hps.get('learning_rate')}")
print(f"dropout_rate: {best_hps.get('dropout_rate')}")


Trial 60 Complete [00h 00m 23s]
val_mse: 0.5946145057678223

Best val_mse So Far: 0.39134538173675537
Total elapsed time: 00h 24m 29s
 48/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

c:\Users\Rafael\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Training R2: 0.7759053814214373, Test R2: 0.6533567308379751
Training MSE: 0.3001754105209001, Test MSE: 0.4511032056307784
Training MAE: 0.4020461176081125, Test MAE: 0.487991071521595
Best hyperparameters:
neurons1: 256
neurons2: 128
neurons3: 64
learning_rate: 0.0001
dropout_rate: 0.0


In [12]:
from tensorflow import keras

neurons1 = 256
neurons2 = 128
neurons3 = 64
learning_rate = 0.0001
dropout_rate = 0.0

model = keras.models.Sequential([
    keras.layers.Dense(neurons1, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons2, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons3, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mse",
    metrics=["mse"]
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")


Epoch 1/100


c:\Users\Rafael\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 32.6124 - mse: 32.6124 - val_loss: 1.9872 - val_mse: 1.9872
Epoch 2/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6799 - mse: 1.6799 - val_loss: 1.1818 - val_mse: 1.1818
Epoch 3/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9911 - mse: 0.9911 - val_loss: 1.0854 - val_mse: 1.0854
Epoch 4/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9617 - mse: 0.9617 - val_loss: 1.0208 - val_mse: 1.0208
Epoch 5/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9113 - mse: 0.9113 - val_loss: 0.9828 - val_mse: 0.9828
Epoch 6/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8448 - mse: 0.8448 - val_loss: 0.9650 - val_mse: 0.9650
Epoch 7/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8608 - mse: 0.8608 - val_loss: 0.9393 - val_mse: 0.9393
Epoch 8/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8155 - mse: 0.8155 - val_loss: 0.9209 - val_mse: 0.9209
Epoch 9/100
152/152 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - los

In [18]:
# The following applicability domains use variables produced from the ANN code above.

ad = TopKatApplicabilityDomain()
ad.fit(X_train)

inside_ad_train = ad.contains(X_train)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.758136332939381, Test R2: 0.6527584260020327
Training MSE: 0.3074352962647974, Test MSE: 0.4274571015772888
Training MAE: 0.40378848239698134, Test MAE: 0.47143375447604696


In [19]:
# The following applicability domains use variables produced from the ANN code above.

ad = KNNApplicabilityDomain()
ad.fit(X_train)

inside_ad_train = ad.contains(X_train)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.7716420827701251, Test R2: 0.6563263569442341
Training MSE: 0.311985795988654, Test MSE: 0.4616764034674082
Training MAE: 0.4047105621657717, Test MAE: 0.48835161900927826


In [20]:
import numpy as np
from scipy.stats import zscore
from tensorflow import keras
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

z_scores = np.abs(zscore(y_train))
mask = (z_scores < 3)
X_train_clean = X_train[mask]
y_train_clean = y_train[mask]

print(f"Removed {len(X_train) - len(X_train_clean)} outliers from training set.")

neurons1 = 256
neurons2 = 128
neurons3 = 64
learning_rate = 0.0001
dropout_rate = 0.0

model = keras.models.Sequential([
    keras.layers.Dense(neurons1, activation="relu", input_shape=(X_train_clean.shape[1],)),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons2, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons3, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mse",
    metrics=["mse"]
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_clean, y_train_clean,
    epochs=100,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

y_pred_train = model.predict(X_train_clean)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train_clean, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mse_train = mean_squared_error(y_train_clean, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train_clean, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")


Removed 74 outliers from training set.
Epoch 1/100


c:\Users\Rafael\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 29.9860 - mse: 29.9860 - val_loss: 1.8774 - val_mse: 1.8774
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4540 - mse: 1.4540 - val_loss: 1.0691 - val_mse: 1.0691
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9360 - mse: 0.9360 - val_loss: 0.9906 - val_mse: 0.9906
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8956 - mse: 0.8956 - val_loss: 0.9496 - val_mse: 0.9496
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8183 - mse: 0.8183 - val_loss: 0.9136 - val_mse: 0.9136
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8320 - mse: 0.8320 - val_loss: 0.8998 - val_mse: 0.8998
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7924 - mse: 0.7924 - val_loss: 0.8709 - val_mse: 0.8709
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7913 - mse: 0.7913 - val_loss: 0.8522 - val_mse: 0.8522
Epoch 9/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los

In [ ]:
# The following applicability domains use variables produced from the ANN code above.

ad = TopKatApplicabilityDomain()
ad.fit(X_train_clean)

inside_ad_train = ad.contains(X_train_clean)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train_clean[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.7312894194131043, Test R2: 0.6207344283340006
Training MSE: 0.30192795675673206, Test MSE: 0.43512580274836554
Training MAE: 0.4014599406236926, Test MAE: 0.47722860130077155


In [22]:
# The following applicability domains use variables produced from the ANN code above.

ad = KNNApplicabilityDomain()
ad.fit(X_train_clean)

inside_ad_train = ad.contains(X_train_clean)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train_clean[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.744966535014033, Test R2: 0.6252611886164933
Training MSE: 0.3029860494465483, Test MSE: 0.5034080156420836
Training MAE: 0.40083283470695846, Test MAE: 0.5063258336105685


In [24]:
import numpy as np
from scipy.stats import zscore
from tensorflow import keras
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd

y_train_series = pd.Series(y_train)

Q1 = y_train_series.quantile(0.25)
Q3 = y_train_series.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

mask = (y_train_series >= lower_bound) & (y_train_series <= upper_bound)
X_train_clean = X_train[mask]
y_train_clean = y_train[mask]

print(f"Removed {len(X_train) - len(X_train_clean)} outliers using IQR method.")


neurons1 = 256
neurons2 = 128
neurons3 = 64
learning_rate = 0.0001
dropout_rate = 0.0

model = keras.models.Sequential([
    keras.layers.Dense(neurons1, activation="relu", input_shape=(X_train_clean.shape[1],)),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons2, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(neurons3, activation="relu"),
    keras.layers.Dropout(dropout_rate),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mse",
    metrics=["mse"]
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_mse',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_clean, y_train_clean,
    epochs=100,
    validation_split=0.2,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

y_pred_train = model.predict(X_train_clean)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train_clean, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)
mse_train = mean_squared_error(y_train_clean, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train_clean, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")


Removed 93 outliers using IQR method.
Epoch 1/100


c:\Users\Rafael\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 30.1157 - mse: 30.1157 - val_loss: 1.9512 - val_mse: 1.9512
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.5382 - mse: 1.5382 - val_loss: 1.0824 - val_mse: 1.0824
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9591 - mse: 0.9591 - val_loss: 0.9907 - val_mse: 0.9907
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8624 - mse: 0.8624 - val_loss: 0.9386 - val_mse: 0.9386
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8434 - mse: 0.8434 - val_loss: 0.9124 - val_mse: 0.9124
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8155 - mse: 0.8155 - val_loss: 0.8911 - val_mse: 0.8911
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8154 - mse: 0.8154 - val_loss: 0.8826 - val_mse: 0.8826
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7860 - mse: 0.7860 - val_loss: 0.8557 - val_mse: 0.8557
Epoch 9/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - los

In [25]:
# The following applicability domains use variables produced from the ANN code above.

ad = TopKatApplicabilityDomain()
ad.fit(X_train_clean)

inside_ad_train = ad.contains(X_train_clean)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train_clean[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.7134630679178048, Test R2: 0.61411902746764
Training MSE: 0.3149858968824533, Test MSE: 0.4433140671241943
Training MAE: 0.41724204220077343, Test MAE: 0.4925239230498098


In [ ]:
# The following applicability domains use variables produced from the ANN code above.

ad = KNNApplicabilityDomain()
ad.fit(X_train)

inside_ad_train = ad.contains(X_train)
inside_ad_test = ad.contains(X_test)

y_train_inside = y_train[inside_ad_train]
y_pred_train_inside = y_pred_train[inside_ad_train]

y_test_inside = y_test[inside_ad_test]
y_pred_test_inside = y_pred_test[inside_ad_test]

r2_train = r2_score(y_train_inside, y_pred_train_inside)
r2_test = r2_score(y_test_inside, y_pred_test_inside)
mse_train = mean_squared_error(y_train_inside, y_pred_train_inside)
mse_test = mean_squared_error(y_test_inside, y_pred_test_inside)
mae_train = mean_absolute_error(y_train_inside, y_pred_train_inside)
mae_test = mean_absolute_error(y_test_inside, y_pred_test_inside)

print(f"Training R2: {r2_train}, Test R2: {r2_test}")
print(f"Training MSE: {mse_train}, Test MSE: {mse_test}")
print(f"Training MAE: {mae_train}, Test MAE: {mae_test}")

Training R2: 0.7716420827701251, Test R2: 0.6563263569442341
Training MSE: 0.311985795988654, Test MSE: 0.4616764034674082
Training MAE: 0.4047105621657717, Test MAE: 0.48835161900927826
